In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')


Reading data

In [2]:
df_infobase = pd.read_csv("../../data/raw/info_base_games.csv",low_memory=False)
df_gamalytic = pd.read_csv("../../data/raw/gamalytic_steam_games.csv",low_memory=False)


Exploring data

In [3]:
df_infobase.dtypes
df_gamalytic.dtypes
print(df_infobase.shape)
print(df_gamalytic.shape)

(99167, 10)
(93338, 6)


In [4]:
#df_infobase.head(10)
#df_infobase.tail(10)
#df_infobase.info()
#df_infobase.describe()
#df_infobase.isnull().sum()
#print(((df_infobase.isnull().sum())/df_infobase.shape[0]) * 100)

Merging 2 csv files info_base and gamalytic steam games 

In [5]:
common_column = 'appid'
df_gamalytic.rename(columns={'steamId': common_column}, inplace=True)
# convert data type object to number for column appid in infobase 
#df_infobase[common_column] = df_infobase[common_column].astype(int) # I assume it's primary key so it will not have any null val

# First Remove any rows if appid is not numeric
df_infobase = df_infobase[pd.to_numeric(df_infobase[common_column], errors='coerce').notna()] 
df_infobase[common_column] = df_infobase[common_column].astype(int)
#print(df_infobase.dtypes)
#print(df_infobase.shape)
def merge_steam_games(x,y):
    df = pd.merge(x,y,on=common_column, how='inner')
    return df 
df = merge_steam_games(df_infobase,df_gamalytic).copy()

Preprocessing release date feature

In [6]:
#df
#print(df.shape)
#df.describe()

df['release_date'].isnull().sum()
#(df['release_date'] == "Coming soon").sum() #636 samples 
#(df['release_date'] == "to be announced").sum() # 0 samples


np.int64(2)

In [7]:
#If there are no parentheses in pattern, .groups() returns an empty tuple.
df.dropna(subset=['release_date'],inplace=True)
df['release_date'] = df['release_date'].astype(str).str.strip().str.lower() # to remove any leading or trailing spaces if exist

unknown_release_dates = [
    r'^coming soon$',
    r'^to be announced$',
    r'^\s*$',  # empty string
    r'^q[1-4] \d{4}$', #q2 2025
    r'^\d{4}$'  # year only
]
def has_known_release(x):
    for pattern in unknown_release_dates:
        if re.match(pattern, x):
            return 0
        return 1
df['is_release_date_known'] = df['release_date'].apply(has_known_release)

df['is_upcoming'] = df['release_date'].str.contains(r'^\d{4}|\bq[1-4]') & (pd.to_datetime(df['release_date'], errors='coerce') > pd.Timestamp.now()) # if condition is true so all of this evaluates to 1 if not evaluates zero for each sample
df['is_upcoming'] = df['is_upcoming'].astype(int)

def preprocess_release_date(x):
    x = str(x)
    if re.search(r'(\d{1,2}) .*? ([A-Za-z]{3}) .*? (\d{2,4})', x):
        day, month, year = re.search(r'(\d{1,2}) .*? ([A-Za-z]{3}) .*? (\d{2,4})', x).groups()
        return f"{day} {month} {year}"
    elif re.search(r'([A-Za-z]{3}) .*? (\d{2,4})', x):
        month, year = re.search(r'([A-Za-z]{3}) .*? (\d{2,4})', x).groups()
        return f"15 {month} {year}"
    elif re.search(r'(\d{1,2}) .*? ([A-Za-z]{3})', x): # DAY MON 
        return np.nan
    elif re.search(r'\bq[1-4] \d{4}\b', x):
        q, y = x.split() 
        quarter_map = {
            'q1': '5 Feb',
            'q2': '5 May',
            'q3': '5 Aug',
            'q4': '5 Nov',
        }
        return f"{quarter_map[q]} {y}"
    elif re.fullmatch(r'\b[\d]{4}\b', x):
        #print(f"{x}")
        year = int(x)
        current_year = pd.Timestamp.now().year
        if(current_year < year):
            return "10 Jun " + str(x)
        else:
            return "1 JAN " + str(x)
    else: # 'to be announced' , 'coming soon' , ' ' , ...
        return np.nan


# for each value in column release date we will apply this function to it 
df['release_date'] = df['release_date'].apply(preprocess_release_date)


In [8]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce') 

df['year'] = df['release_date'].dt.year.fillna(0).astype(int)
fraction_of_year = np.where(df['is_release_date_known'], (df['release_date'].dt.dayofyear - 1) / 365, 0)
df['sin_day'] = np.sin(2 * np.pi * fraction_of_year)
df['cos_day'] = np.cos(2 * np.pi * fraction_of_year)
df['sin_day'].fillna(0, inplace=True)
df['cos_day'].fillna(0, inplace=True)
df.drop('release_date',axis=1,inplace=True)


Checking about missing data and new features

In [9]:
#print(df['is_release_date_known'].shape)
#print(df['year'].shape)
#print(df['sin_day'].shape)
#print(df['cos_day'].shape)
print(df['is_release_date_known'].isnull().sum())
print(df['year'].isnull().sum())
print(df['sin_day'].isnull().sum())
print(df['cos_day'].isnull().sum())
print(df['is_upcoming'].isnull().sum())

0
0
0
0
0


Preprocessing Achievement_total feature
Note: There is relation between it and steam achievement

In [10]:
df['achievements_total'].isnull().sum()/df.shape[0] #0.46

# Convert 'Total achievement' to number, and if there errors set them null
df['achievements_total'] = pd.to_numeric(df['achievements_total'], errors='coerce')

# if total achievement has value and steam achievement is false so convert it into true 
df.loc[(df['achievements_total'] > 0) & (df['steam_achievements'] == False), 'steam_achievements'] = True

# if steam achievement is false and achievement total is null so set it zero 
df.loc[df['steam_achievements'] == False, 'achievements_total'] = 0

# using KNN Imputer for fill null values of total achievement if steam achievement is true 
mask = (df['steam_achievements'] == True)

rows = df.loc[mask, ['achievements_total']]

imputer = KNNImputer(n_neighbors=5)
imputed = imputer.fit_transform(rows)
df.loc[mask, 'achievements_total'] = np.floor(imputed)

# another option you can use median 
#median_val = df.loc[df['steam_achievements'] == True, 'achievements_total'].median()
#df.loc[(df['steam_achievements'] == True) & (df['achievements_total'].isna()), 'achievements_total'] = median_val

# convert boolean to int (0,1)
df['steam_achievements'] = df['steam_achievements'].astype(int)
#it takes 6.8 sec cuz of knn imputer


In [11]:
print(df['achievements_total'].isnull().sum())
df

0


,appid,name,metacritic,steam_achievements,steam_trading_cards,workshop_support,genres,achievements_total,supported_platforms,price,copiesSold,publisherClass,reviewScore,aiContent,is_release_date_known,is_upcoming,year,sin_day,cos_day
0,2574000,Femboy Burgers,NaN,1,True,True,"Casual, Indie",40.0,"['windows', 'mac', 'linux']",19.99,1727,Indie,88,NaN,1,0,2023,-0.974100,0.226116
1,2574120,PPA Pickleball Tour 2025,NaN,1,True,True,"Indie, Simulation, Sports",18.0,"['windows', 'mac', 'linux']",29.99,367,Indie,38,NaN,1,0,2024,-0.230306,-0.973118
2,2573200,Squeaky Squad,NaN,1,True,True,"Action, Adventure, Indie",27.0,"['windows', 'mac', 'linux']",0.00,1298,Hobbyist,82,NaN,1,0,2024,0.956235,0.292600
3,2569520,Naturpark Lillebælt VR,NaN,1,True,True,"Action, Adventure",40.0,"['windows', 'mac', 'linux']",0.00,65,Hobbyist,75,NaN,1,0,2023,-0.958718,-0.284359
4,2567670,A Lively Haunt,NaN,1,True,True,"Adventure, Indie",40.0,"['windows', 'mac', 'linux']",9.99,47,Indie,100,NaN,1,0,2023,-0.958718,-0.284359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69423,1465460,Infection Free Zone,NaN,1,False,True,"Action, Indie, Simulation, Strategy, Early Access",20.0,['windows'],24.99,647531,Indie,79,NaN,1,0,2024,0.972118,-0.234491
69424,2999030,Exploding Kittens® 2,NaN,1,False,False,"Action, Casual, Indie, Simulation, Strategy",11.0,['windows'],12.49,7749,Indie,72,NaN,1,0,2024,-0.693281,-0.720667
69425,1548850,Six Days in Fallujah,NaN,1,False,False,"Action, Early Access",34.0,['windows'],39.99,633920,Indie,83,NaN,1,0,2023,0.296713,-0.954967
69426,2054150,Tower Defender VR: Last Adventure,NaN,0,False,False,"Casual, Indie, RPG, Strategy",0.0,['windows'],19.99,100,Hobbyist,92,NaN,1,0,2022,-0.213521,-0.976938


#### Final Thoughts: 

---

**Preprocessing Release Date**

I have many approaches 

*First approach*

just consider only "year" column and ignore day and month 

---

*Second approach*

Create new column for binary feature is_release_date_known
1 indicates a specific date or quarter is known.
0 indicates the entry is "to be announced" or "coming soon".
For the main release date feature (fractional_year) = year + (day of year - 1)/days in year

---

*Third approach*

guide : https://medium.com/%40paghadalsneh/handling-date-and-time-data-in-machine-learning-a-comprehensive-guide-5d30141cbfec 

splitting date into 3 features (day,month,year)

---

*Fourth approach* 

using Cyclical encoding 

---

***Fifth Approach*** 

Creating two boolean features 

**is_release_date_known** : A missingness indicator set to 1 if game has a specific past or present date (e.g. “15 Apr 2024” or “2023-08-05”), and 0 if (“coming soon”, “to be announced”, “Q[1–4] YYYY”, or “YYYY” only):

**is_upcoming** : A flag set to 1 if a game release date is strictly after the current date, and 0 otherwise.

I think two booleans together will make good combination 

Creating "year" , "fraction_of_year_sin" , "fraction_of_year_cos" features 
so Here we applied Cyclical encoding for not missing seasonal patterns 

There are many updates we can do but I don't know it will help or not, We need to select features first to be able to test
like in Q[1-4] yyyy or yyyy only 
we can fill it by mode of month and median of days, and maybe in each quarter calculate mean or use imputers



---

**Preprocessing total achievement**

I recommend making web scraping , it will be better and more accurate for sure instead of filling nulls by with mean/median/imputers 

but We must put into consideration Is it possible or not?
